In [ ]:
# test fc 
import torch 
from tests.datagen import save_tensor_to_csv
import os
xh = 3
xw = 4
th = 2
inner = 5
torch.set_printoptions(precision=10, linewidth=2000)
torch.random.manual_seed(20)
W0 = torch.nn.Parameter(torch.randn(inner, xh) )
b0 = torch.nn.Parameter(torch.randn(inner, 1) )
W1 = torch.nn.Parameter(torch.randn(th, inner) )
b1 = torch.nn.Parameter(torch.randn(th, 1) )

x = torch.ones(xh, xw)
y1 = torch.mm(W0, x) + b0
z1 = torch.sigmoid(y1)
y2 = torch.mm(W1, z1) + b1
z2 = torch.tanh(y2)

os.remove("static_data/fc.txt")
save_tensor_to_csv(W0, "static_data/fc.txt")
save_tensor_to_csv(b0, "static_data/fc.txt")
save_tensor_to_csv(W1, "static_data/fc.txt")
save_tensor_to_csv(b1, "static_data/fc.txt")

exp = torch.exp(z2)
s = exp / exp.sum(dim=0, keepdim=True)

# cross entropy:
t = torch.ones_like(s) * 5
#print("t: ", t, "\ns: ", s)
ce = -(t * torch.log(s)).sum()
#ce = (s - t).pow(2).mean()
ce.backward()
print("error: " , ce)

save_tensor_to_csv(W0.grad, "static_data/fc.txt")
save_tensor_to_csv(b0.grad, "static_data/fc.txt")
save_tensor_to_csv(W1.grad, "static_data/fc.txt")
save_tensor_to_csv(b1.grad, "static_data/fc.txt")

In [ ]:
# test attention
import torch
import os
import math
from tests.datagen import save_tensor_to_csv
torch.manual_seed(10)
torch.set_printoptions(precision=8, linewidth=2000)

Ei = 64 # input embedding size
Eq = 9  # query embedding size
Ek = Eq # key embedding size
Ev = Ei # value , i.e. output embedding size
S = 10 # seq_len

Q = torch.nn.Parameter(torch.randn(Eq, Ei) * math.sqrt(1/Ei))
K = torch.nn.Parameter(torch.randn(Ek, Ei) * math.sqrt(1/Ei))
V = torch.nn.Parameter(torch.randn(Ev, Ei) * math.sqrt(1/Ei))

q = torch.rand(S, Ei)
k = torch.rand(S, Ei)
v = torch.rand(S, Ei)

q_ = q @ Q.t()
k_ = k @ K.t()
v_ = v @ V.t()

q_.retain_grad()
k_.retain_grad()
v_.retain_grad()

print("Q.shape: ", Q.shape, "K.shape: ", K.shape, "V.shape: ", V.shape)
print("q_.shape: ", q_.shape, "k_.shape: ", k_.shape, "v_.shape: ", v_.shape)

qkt = q_ @ k_.t() / (Eq ** .5)
print("qkt.shape: ", qkt.shape)
qkt.retain_grad()
smax = qkt.exp() / qkt.exp().sum(dim=1, keepdim=True)
smax.retain_grad()
output = smax @ v_
output.retain_grad()
print("otuput shape: ", output.shape)

t = torch.ones_like(output)
loss = (t - output).pow(2).mean()
loss.backward()

l2_grad = 2 * (output - t) / (S * Ev)

qkt_grad_in = torch.mm(smax.t(), l2_grad)

v_grad_in = torch.mm(smax.t(), l2_grad)
q_grad_in = torch.mm(qkt.grad, k_) / (Eq ** .5)
k_grad_in = torch.mm(q_.t(), qkt.grad).t() / (Eq ** .5)

Q_grad_in = torch.mm(q.t(), q_grad_in).t()
K_grad_in = torch.mm(k.t(), k_grad_in).t()
V_grad_in = torch.mm(v.t(), v_grad_in).t()

assert(torch.allclose(v_.grad, v_grad_in))
assert(torch.allclose(q_.grad, q_grad_in))
assert(torch.allclose(k_.grad, k_grad_in))
assert(torch.allclose(Q.grad, Q_grad_in))
assert(torch.allclose(K.grad, K_grad_in))
assert(torch.allclose(V.grad, V_grad_in))

file = "static_data/attention.txt"
try:
    os.remove(file)
except:
    pass
save_tensor_to_csv(Q, file)
save_tensor_to_csv(K, file)
save_tensor_to_csv(V, file)
save_tensor_to_csv(q, file)
save_tensor_to_csv(k, file)
save_tensor_to_csv(v, file)
save_tensor_to_csv(qkt, file)
save_tensor_to_csv(smax, file)
save_tensor_to_csv(output, file)
save_tensor_to_csv(Q.grad, file)
save_tensor_to_csv(K.grad, file)
save_tensor_to_csv(V.grad, file)

# print("output.grad:\n ", output.grad)
# print("smax.grad:\n ", smax.grad)
# print("qkt.grad:\n ", qkt.grad)
# print("q.grad:\n ", q_.grad)
# print("k.grad:\n ", k_.grad)
# print("v.grad:\n ", v_.grad)

Q.shape:  torch.Size([8, 64]) K.shape:  torch.Size([8, 64]) V.shape:  torch.Size([64, 64])
q_.shape:  torch.Size([10, 8]) k_.shape:  torch.Size([10, 8]) v_.shape:  torch.Size([10, 64])
qkt.shape:  torch.Size([10, 10])
otuput shape:  torch.Size([10, 64])


'static_data/attention.txt'

In [ ]:
# test linear
import torch 
from tests.datagen import save_tensor_to_csv
import os
s = 3
e = 4
l0 = 5
l1 = 6

torch.set_printoptions(precision=10, linewidth=2000)
torch.random.manual_seed(20)
W0 = torch.nn.Parameter(torch.randn(l0, e) )
W1 = torch.nn.Parameter(torch.randn(l1, l0) )
x = torch.randn(s, e)

l0 = torch.mm(x, W0.t())
l1 = torch.mm(l0, W1.t())

file = "static_data/linear.txt"
try:
    os.remove(file)
except:
    pass

print(l1.shape)
print("W0.shape: ", W0.shape, "W1.shape: ", W1.shape, "x.shape: ", x.shape)

loss = (l1 - torch.ones_like(l1)).pow(2).mean()
loss.backward()

print("Loss: ", loss, "\nW0.grad: ", W0.grad, "\nW1.grad: ", W1.grad)

save_tensor_to_csv(W0, file)
save_tensor_to_csv(W1, file)
save_tensor_to_csv(x, file)
save_tensor_to_csv(W0.grad, file)
save_tensor_to_csv(W1.grad, file)

In [79]:
qkt = torch.tensor([
[ 0.076297254824, -0.089440519897, -0.371014912449, -0.189202182827, -0.298909048615, -0.515861427292, -0.142748949398, -0.158466049148, -0.288262806786, -0.072505993565],
[ 0.255754932347,  0.019554433842, -0.057004364579,  0.015881522730, -0.024455754024, -0.322306133226,  0.082900243477, -0.017978623898,  0.059798919489,  0.016707988731],
[ 0.273374604582,  0.037210046839,  0.144808477272,  0.121815029922,  0.238055286014, -0.068983065517,  0.155575745483, -0.014508721423, -0.037470733306,  0.131346675512],
[ 0.238488928187, -0.010167704091,  0.305556417810,  0.167380170770,  0.310425327521,  0.027609696010,  0.100895234642, -0.026971487774, -0.058357885789,  0.088043750426],
[-0.168152623492, -0.281570069192, -0.499293169337, -0.246570567285, -0.348227687118, -0.224205006938, -0.219531556660, -0.343482802825, -0.369890804012, -0.169075230100],
[ 0.260511982217,  0.006776621054, -0.095077215252,  0.147110187039,  0.068072541528,  0.010974141683,  0.309812708134, -0.003253084334, -0.007544979050,  0.325015241553],
[ 0.291577117281,  0.005927566585, -0.125921526200,  0.068507524355,  0.002718956754, -0.215235421407,  0.151158941719,  0.007180936854,  0.059886240064,  0.195653552867],
[ 0.247014384165,  0.002506488994, -0.074987019181,  0.013352291769,  0.015410459461, -0.119976157293,  0.186167924457, -0.025999013549, -0.035065335487,  0.096228932996],
[ 0.273534097540, -0.036932532615, -0.032342263459,  0.102535537908,  0.036252774652, -0.140272732616,  0.215586435001, -0.085631690456, -0.005099915857,  0.142097005595],
[ 0.429310777327,  0.123783313888,  0.029616672762,  0.095774946593,  0.275906439866, -0.034831717230,  0.423534844756,  0.072774490637, -0.017288310922,  0.200044019217]])

In [81]:
print(qkt.shape)
s = qkt.sum(1)
print(s)
print(s.max())
print(s.min())


torch.Size([10, 10])
tensor([-2.05011463,  0.02885318,  0.98122334,  1.14290249, -2.86999941,  1.02239823,  0.44145387,  0.30465296,  0.46972671,  1.59862554])
tensor(1.59862554)
tensor(-2.86999941)
